In [1]:
import pandas as pd
#test_sets = <input data>  ==> string list 

#data1
df = pd.read_csv("./Reviews.csv")
df = df[df['Score'] != 3]
df['sentiment'] = df['Score'].apply(lambda rating : 'positive' if rating > 3 else 'negative')
review_df1=df[['Text','sentiment']]
test_sets1=review_df1.Text.values


#data2
df = pd.read_csv("./Tweets.csv")
review_df=df[['text','airline_sentiment']]
review_df2 = review_df[review_df['airline_sentiment'] != 'neutral']
review_df2.columns=["Text","sentiment"]
test_sets2=review_df2.Text.values
answers=review_df2.sentiment.values

# data concatanation
review_df=pd.concat([review_df1,review_df2],ignore_index=True)
print(review_df)
sentiment_label = review_df.sentiment.factorize()
test_sets=review_df.Text.values


                                                     Text sentiment
0       I have bought several of the Vitality canned d...  positive
1       Product arrived labeled as Jumbo Salted Peanut...  negative
2       This is a confection that has been around a fe...  positive
3       If you are looking for the secret ingredient i...  negative
4       Great taffy at a great price.  There was a wid...  positive
...                                                   ...       ...
537350  @AmericanAir my flight was Cancelled Flightled...  negative
537351         @AmericanAir right on cue with the delays👌  negative
537352  @AmericanAir thank you we got on a different f...  positive
537353  @AmericanAir leaving over 20 minutes Late Flig...  negative
537354  @AmericanAir you have my money, you change my ...  negative

[537355 rows x 2 columns]


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(test_sets)
# print(tokenizer.word_index, len(tokenizer.word_index))


In [3]:
encoded_docs = tokenizer.texts_to_sequences(test_sets)
# print(encoded_docs)

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=200)
print(padded_sequence)

[[    0     0     0 ...   101    58   141]
 [    0     0     0 ...    38    24  5778]
 [    0     0     0 ...     5     1 13888]
 ...
 [    0     0     0 ...  1161     5  4306]
 [    0     0     0 ...   751   753 19091]
 [    0     0     0 ...    96    13  8104]]


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

vocab_size=len(tokenizer.word_index)+1
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

2021-11-30 08:29:20.888851: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-11-30 08:29:20.888882: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-30 08:29:20.889655: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           4231168   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 4,247,819
Trainable params: 4,247,819
Non-trainable params: 0
______________________________________________

In [8]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=1, batch_size=512)


840/840 [==============================] - 359s 424ms/step - loss: 0.2284 - accuracy: 0.9122 - val_loss: 0.2521 - val_accuracy: 0.8897


In [12]:
epochs=6
batch_size=64
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=epochs, batch_size=batch_size)
model.save('trained_model(epoch='+str(epochs)+',batch_size='+str(batch_size)+').h5')

Epoch 1/6
6717/6717 [==============================] - 2153s 320ms/step - loss: 0.1728 - accuracy: 0.9347 - val_loss: 0.3036 - val_accuracy: 0.8897
Epoch 2/6
6717/6717 [==============================] - 2152s 320ms/step - loss: 0.1420 - accuracy: 0.9467 - val_loss: 0.2818 - val_accuracy: 0.8960
Epoch 3/6
6717/6717 [==============================] - 2164s 322ms/step - loss: 0.1222 - accuracy: 0.9550 - val_loss: 0.2790 - val_accuracy: 0.9012
Epoch 4/6
6717/6717 [==============================] - 2162s 322ms/step - loss: 0.1105 - accuracy: 0.9594 - val_loss: 0.2680 - val_accuracy: 0.9056
Epoch 5/6
6717/6717 [==============================] - 2160s 322ms/step - loss: 0.1015 - accuracy: 0.9630 - val_loss: 0.2907 - val_accuracy: 0.9054
Epoch 6/6
6717/6717 [==============================] - 2159s 321ms/step - loss: 0.0949 - accuracy: 0.9656 - val_loss: 0.2887 - val_accuracy: 0.9086


In [6]:
epochs=6
batch_size=32
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=epochs, batch_size=batch_size)
model.save('trained_model(epoch='+str(epochs)+',batch_size='+str(batch_size)+').h5')

Epoch 1/6
13434/13434 [==============================] - 4059s 302ms/step - loss: 0.1969 - accuracy: 0.9244 - val_loss: 0.2622 - val_accuracy: 0.8953
Epoch 2/6
13434/13434 [==============================] - 4045s 301ms/step - loss: 0.1399 - accuracy: 0.9472 - val_loss: 0.2430 - val_accuracy: 0.9078
Epoch 3/6
13434/13434 [==============================] - 4040s 301ms/step - loss: 0.1197 - accuracy: 0.9558 - val_loss: 0.2668 - val_accuracy: 0.9074
Epoch 4/6
13434/13434 [==============================] - 4036s 300ms/step - loss: 0.1091 - accuracy: 0.9602 - val_loss: 0.2642 - val_accuracy: 0.9065
Epoch 5/6
13434/13434 [==============================] - 4034s 300ms/step - loss: 0.1018 - accuracy: 0.9630 - val_loss: 0.2744 - val_accuracy: 0.9118
Epoch 6/6
13434/13434 [==============================] - 4019s 299ms/step - loss: 0.0969 - accuracy: 0.9648 - val_loss: 0.2701 - val_accuracy: 0.9112


In [7]:
epochs=7
batch_size=128
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=epochs, batch_size=batch_size)
model.save('trained_model(epoch='+str(epochs)+',batch_size='+str(batch_size)+').h5')

Epoch 1/7
3359/3359 [==============================] - 1140s 339ms/step - loss: 0.0873 - accuracy: 0.9687 - val_loss: 0.2888 - val_accuracy: 0.9108
Epoch 2/7
3359/3359 [==============================] - 1141s 340ms/step - loss: 0.0837 - accuracy: 0.9701 - val_loss: 0.2867 - val_accuracy: 0.9133
Epoch 3/7
3359/3359 [==============================] - 1140s 339ms/step - loss: 0.0805 - accuracy: 0.9709 - val_loss: 0.3047 - val_accuracy: 0.9123
Epoch 4/7
3359/3359 [==============================] - 1139s 339ms/step - loss: 0.0779 - accuracy: 0.9722 - val_loss: 0.3332 - val_accuracy: 0.9095
Epoch 5/7
3359/3359 [==============================] - 1139s 339ms/step - loss: 0.0757 - accuracy: 0.9730 - val_loss: 0.3263 - val_accuracy: 0.9099
Epoch 6/7
3359/3359 [==============================] - 1138s 339ms/step - loss: 0.0733 - accuracy: 0.9739 - val_loss: 0.3161 - val_accuracy: 0.9102
Epoch 7/7
3359/3359 [==============================] - 1126s 335ms/step - loss: 0.0710 - accuracy: 0.9749 - val_